In [30]:
import folium
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *

#importa os dados de colheitas e dados dos csvs
data_cidades = pd.read_csv("dados_cidades.csv")
data_colheitas = pd.read_csv("dados_colheitas.csv")

In [32]:
def atualiza_mapa(colheita):
  result = {}
  m = folium.Map(location=(-22.3158656,-49.0713603), zoom_start=6)

  for index, row_colheita in data_colheitas.iterrows():
    # percorre cada colheita até encontrar a que o usuário escolheu(acho que
    # existem formas de melhorar essa pesquisa)
    if(colheita == row_colheita["COLHEITA"]):
      for index, row_cidade in data_cidades.iterrows():
        # se a cidade atual tem uma temperatura ou pluviometria dentro da faixa
        # esperada, adicionamos a lista de resultados
        if (float(row_cidade["MEDIA TEMP."].replace(',', '.')) >= (float(row_colheita["TEMP IDEAL"].replace(',', '.')) - 1.5)
        and float(row_cidade["MEDIA TEMP."].replace(',', '.')) <= (float(row_colheita["TEMP IDEAL"].replace(',', '.')) + 1.5)
        and float(row_cidade["PRECIPITAÇÃO"].replace(',', '.')) <= (float(row_colheita["PLUVIOMETRIA IDEAL"].replace(',', '.')) + 200)
        and float(row_cidade["PRECIPITAÇÃO"].replace(',', '.')) >= (float(row_colheita["PLUVIOMETRIA IDEAL"].replace(',', '.')) - 200)):
          result.setdefault(row_cidade.CIDADE, []).append(row_cidade)


  for cidade, info in result.items():
    popupStr = ""
    popupColor = ""
    popupLoc = info[0].LOCALIZAÇÃO.split(",")
    # para cada cidade adequada, adicionamos o marcador
    if len(info) == 1 :
      popupStr += ("<b>" + info[0].CIDADE + "</b> é adequado pois:<br><b>" + info[0].ESTAÇÃO \
        + "</b> tem <b>Temperatura Média</b> de " + info[0]["MEDIA TEMP."] \
        + "<br> e <b>Pluviosidade</b> de " + info[0].PRECIPITAÇÃO)
      popupColor = "lightgreen"

    elif len(info) > 1:
      score = 0
      popupStr += "<b>" + info[0].CIDADE + "</b> é adequado pois:"
      for occur in info:
        score+=1
        popupStr += ("<br><b>" + occur.ESTAÇÃO \
          + "</b> tem <b>Temperatura Média</b> de " + occur["MEDIA TEMP."] \
          + "<br> e <b>Pluviosidade</b> de " + occur.PRECIPITAÇÃO)
      if score <= 2:
        popupColor = "green"
      else:
        popupColor = "darkgreen"


    folium.Marker(
      location= popupLoc,
      tooltip= "Mais informações!",
      popup= folium.Popup(popupStr, min_width=150, max_width=300),
      icon=folium.Icon(icon="tractor", prefix="fa", color = popupColor),
    ).add_to(m)

  display(m)

def dropdown_handler(change):
    global colheita
    colheita = change.new
    atualiza_mapa(colheita)


In [34]:
widget1 = Dropdown(options = data_colheitas.COLHEITA.unique(), value = None)
widget1.observe(dropdown_handler, names='value')
print("Escolha a colheita que deseja cultivar:")
display(widget1)

Escolha a colheita que deseja cultivar:


Dropdown(options=('Açaí', 'Ameixa', 'Amora', 'Arroz', 'Banana', 'Batata', 'Caju', 'Cana', 'Castanha-do-Brasil'…